# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup (provided in the starter code)
import hvplot.pandas
import pandas as pd
import requests

# Import API key (provided in the starter code)
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame (provided in the starter code)
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.11,88,87,14.84,TF,1737658125
1,1,kindu,-2.9500,25.9500,21.46,99,100,1.68,CD,1737658126
2,2,port lincoln,-34.7333,135.8667,15.61,78,0,3.08,AU,1737658127
3,3,waitangi,-43.9535,-176.5597,16.57,99,99,2.74,NZ,1737658129
4,4,kupang,-10.1667,123.5833,26.32,100,100,0.51,ID,1737658130


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [48]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles= "OSM",
    size="Humidity",
    color="City",
    frame_width=700,
    frame_height=500,
    hover_cols=["Lng", "Lat", "City", "Humidity"]
) 

# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

### The fofflowing are my criteria for what I consider an ideal weather condition.
*I have walking and exploring in mind for this criteria.*

* Temperature: I want the temperature to be between 21 and 27C.
* Wind Speed: Less than 4m/s would be ideal for city exploration as the gentle winds can be comfortable.
* Cloudiness: Less than 25% would be clear skies with a little barrier from the harsh sunlight. 


In [44]:
# Narrow down cities that fit criteria and drop any results with null values
favorable_weather_cities = city_data_df.loc[
    (city_data_df["Max Temp"] >= 21)&
    (city_data_df["Max Temp"] <= 27)&
    (city_data_df["Wind Speed"] < 4)&
    (city_data_df["Cloudiness"] < 25)
]
# Drop any rows with null values
favorable_weather_cities = favorable_weather_cities.dropna()

# Display sample data
favorable_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,98,puerto ayora,-0.7393,-90.3518,26.42,89,13,2.68,EC,1737658240
172,172,cabo san lucas,22.8909,-109.9124,23.17,52,20,3.09,MX,1737658096
316,316,bomi,7.2461,-11.5258,26.37,71,8,3.08,SL,1737658486
336,336,konarka,19.9000,86.1167,22.47,75,6,2.97,IN,1737658510
338,338,puerto deseado,-47.7503,-65.8938,21.70,34,15,1.50,AR,1737658513
366,366,richards bay,-28.7830,32.0377,22.83,93,13,2.61,ZA,1737658544
371,371,mongo,12.1844,18.6930,26.56,14,4,3.60,TD,1737658550
392,392,bucerias,20.7500,-105.3333,23.99,64,0,3.60,MX,1737658573
449,449,acapulco de juarez,16.8634,-99.8901,25.67,66,0,2.76,MX,1737658471
472,472,tindouf,27.6711,-8.1474,22.25,12,0,1.54,DZ,1737658667


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = favorable_weather_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
98,puerto ayora,EC,-0.7393,-90.3518,89,
172,cabo san lucas,MX,22.8909,-109.9124,52,
316,bomi,SL,7.2461,-11.5258,71,
336,konarka,IN,19.9000,86.1167,75,
338,puerto deseado,AR,-47.7503,-65.8938,34,
366,richards bay,ZA,-28.7830,32.0377,93,
371,mongo,TD,12.1844,18.6930,14,
392,bucerias,MX,20.7500,-105.3333,64,
449,acapulco de juarez,MX,16.8634,-99.8901,66,
472,tindouf,DZ,27.6711,-8.1474,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
   "limit":1,
   "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
cabo san lucas - nearest hotel: Comfort Rooms
bomi - nearest hotel: No hotel found
konarka - nearest hotel: Hotel Sun Village
puerto deseado - nearest hotel: Los Olmos
richards bay - nearest hotel: SUN1 Richards Bay
mongo - nearest hotel: Consulat Hotel
bucerias - nearest hotel: Grand Decameron Complex, A Trademark All Inclusive
acapulco de juarez - nearest hotel: Hotel del Valle
tindouf - nearest hotel: محمد بوسبي
tchollire - nearest hotel: No hotel found
ndele - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
lompoc - nearest hotel: Red Roof Inn Lompoc
yanchep - nearest hotel: Yanchep Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
98,puerto ayora,EC,-0.7393,-90.3518,89,Hostal La Mirada De Solitario George
172,cabo san lucas,MX,22.8909,-109.9124,52,Comfort Rooms
316,bomi,SL,7.2461,-11.5258,71,No hotel found
336,konarka,IN,19.9000,86.1167,75,Hotel Sun Village
338,puerto deseado,AR,-47.7503,-65.8938,34,Los Olmos
366,richards bay,ZA,-28.7830,32.0377,93,SUN1 Richards Bay
371,mongo,TD,12.1844,18.6930,14,Consulat Hotel
392,bucerias,MX,20.7500,-105.3333,64,"Grand Decameron Complex, A Trademark All Inclu..."
449,acapulco de juarez,MX,16.8634,-99.8901,66,Hotel del Valle
472,tindouf,DZ,27.6711,-8.1474,12,محمد بوسبي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles= "OSM",
    size="Humidity",
    color="City",
    frame_width=700,
    frame_height=500,
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)